In [21]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNetCV

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error, r2_score, median_absolute_error, mean_absolute_percentage_error

from pickle import dump

In [22]:
total_data = pd.read_csv(r"C:\Users\34616\Documents\4GEEKS\clases\clase20_decisiontrees\decision-trees\data\processed\skin\total_clean.csv")
print(total_data.head())

   Pregnancies  Glucose  BloodPressure  Insulin   BMI  \
0            6      148             72        0  33.6   
1            1       85             66        0  26.6   
2            8      183             64        0  23.3   
3            1       89             66       94  28.1   
4            5      116             74        0  25.6   

   DiabetesPedigreeFunction  Age  Outcome  SkinThickness  
0                     0.627   50        1             35  
1                     0.351   31        0             29  
2                     0.672   32        1              0  
3                     0.167   21        0             23  
4                     0.201   30        0              0  


## model 3 --> predicting skin thickness based on rest of features

In [23]:
total_data.describe()

zeros = total_data[total_data['SkinThickness'] == 0]
enteros = total_data.drop(zeros.index)
print("enteros:")
print(enteros.describe())
print("zeros:")
print(zeros.describe())

X_enteros = enteros.drop('SkinThickness', axis=1)
y_enteros = enteros['SkinThickness']

X_zeros = zeros.drop('SkinThickness', axis=1)
y_zeros = zeros['SkinThickness']


enteros:
       Pregnancies     Glucose  BloodPressure     Insulin         BMI  \
count   518.000000  518.000000     518.000000  518.000000  518.000000   
mean      3.442085  120.216216      71.594595  113.754826   32.796446   
std       3.178677   30.481176      12.319322  119.769442    6.803895   
min       0.000000   56.000000      24.000000    0.000000   18.200000   
25%       1.000000   98.000000      64.000000    0.000000   27.825000   
50%       2.000000  114.000000      72.000000   90.500000   32.800000   
75%       5.000000  140.000000      80.000000  165.000000   36.800000   
max      13.000000  199.000000     110.000000  846.000000   67.100000   

       DiabetesPedigreeFunction         Age     Outcome  SkinThickness  
count                518.000000  518.000000  518.000000     518.000000  
mean                   0.481124   31.343629    0.322394      28.918919  
std                    0.290204   10.397034    0.467845      10.004312  
min                    0.085000   21.0000

In [30]:
X_enteros.head() # type: ignore


,Pregnancies,Glucose,BloodPressure,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,0,33.6,0.627,50,1
1,1,85,66,0,26.6,0.351,31,0
3,1,89,66,94,28.1,0.167,21,0
5,3,78,50,88,31.0,0.248,26,1
6,2,197,70,543,30.5,0.158,53,1


In [33]:
scaler = StandardScaler()

X_enteros_scaled = scaler.fit_transform(X_enteros)
X_enteros_scaled = pd.DataFrame(X_enteros_scaled, columns=X_enteros.columns)
X_zeros_scaled = scaler.transform(X_zeros)
X_zeros_scaled = pd.DataFrame(X_zeros_scaled, columns=X_zeros.columns)

X_enteros_scaled



,Pregnancies,Glucose,BloodPressure,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0.805488,0.912387,0.032940,-0.950700,0.118216,0.503155,1.796128,1.449757
1,-0.769013,-1.156460,-0.454571,-0.950700,-0.911601,-0.448820,-0.033083,-0.689771
2,-0.769013,-1.025105,-0.454571,-0.165100,-0.690926,-1.083471,-0.995825,-0.689771
3,-0.139213,-1.386332,-1.754599,-0.215245,-0.264287,-0.804087,-0.514454,1.449757
4,-0.454113,2.521491,-0.129564,3.587393,-0.337846,-1.114514,2.084951,1.449757
...,...,...,...,...,...,...,...,...
513,1.750189,1.634842,0.195443,-0.950700,1.648230,-0.269463,1.122208,1.449757
514,2.065090,-0.631038,0.357947,0.553640,0.015235,-1.069674,3.047693,-0.689771
515,-0.454113,0.058577,-0.129564,-0.950700,0.588990,-0.486762,-0.418180,-0.689771
516,0.490588,0.025739,0.032940,-0.014666,-0.970447,-0.814434,-0.129357,-0.689771


In [34]:
X_zeros_scaled

,Pregnancies,Glucose,BloodPressure,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,1.435289,2.061747,-0.617074,-0.9507,-1.397086,0.658369,0.063192,1.449757
1,0.490588,-0.138456,0.195443,-0.9507,-1.058717,-0.966199,-0.129357,-0.689771
2,1.435289,0.157094,1.982982,-0.9507,-0.079059,-0.859274,2.181225,1.449757
3,0.175688,-0.335489,1.657975,-0.9507,0.706683,-1.000690,-0.129357,-0.689771
4,2.065090,1.569164,0.195443,-0.9507,0.765530,0.192728,0.255740,1.449757
...,...,...,...,...,...,...,...,...
188,-1.083914,0.091416,0.032940,-0.9507,0.515431,-0.769595,1.988677,1.449757
189,-0.769013,-0.466844,0.357947,-0.9507,0.691971,-0.979995,-0.514454,-0.689771
190,0.805488,2.291619,1.657975,-0.9507,0.397738,-0.700611,3.336516,1.449757
191,1.750189,-1.025105,-0.779578,-0.9507,-1.514779,-1.169701,0.159466,-0.689771


In [36]:
linear_skinpred_model = LinearRegression()
linear_skinpred_model.fit(X_enteros_scaled, y_enteros)

LinearRegression()

In [37]:
skinpred = linear_skinpred_model.predict(X_zeros_scaled)
skinpred

array([21.31354879, 22.35862987, 30.84643214, 33.64572694, 36.11226579,
       25.07700473, 27.29217558, 33.02901905, 37.37883814, 37.14670518,
       30.68640954, 17.03032571, 31.23532911, 36.81282311, 24.90671619,
       25.04845414, 30.99540321, 21.62510826, 30.72894434, 38.33787685,
       42.23592476, 29.75623612, 45.48599853, 14.73030648, 21.86413819,
       34.26310913, 21.63505243, 17.76665259, 34.44200348, 18.09883767,
       29.86658533, 28.83658031, 31.28093961, 29.66170284, 24.75784544,
       28.80699387, 24.48690355, 30.85690354, 26.10274228, 18.26015123,
       31.09279422, 31.27363056, 18.60203476, 45.71663568, 27.1371833 ,
       25.73386872, 27.9491428 , 28.38888955, 28.21664587, 41.60761999,
       35.64637438, 20.40947857, 23.33505278, 24.34047853, 18.72251523,
       28.19402316, 19.81381318, 35.23302065, 35.35063223, 34.99776798,
       28.696627  , 30.66703379, 39.86917893, 31.00049538, 39.7690192 ,
       13.77318189, 22.08599523, 29.3429612 , 29.0274696 , 23.75